In [ ]:
!pip install yfinance

In [ ]:
!pip install pandas_ta

In [41]:
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_datareader.data as web
import pandas_ta as ta
import matplotlib.pyplot as plt
from datetime import date
plt.style.use('fivethirtyeight')
yf.pdr_override()

In [ ]:
stocksymbols = ['HDFCBANK.NS']
startdate = date(2018,4,1)
end_date = date(2019,3,31)
print(end_date)
def getMyPortfolio(stocks = stocksymbols ,start = startdate , end = end_date):
    data = web.get_data_yahoo(stocks , data_source='yahoo' , start = start ,end= end )
    return data

In [ ]:
df = getMyPortfolio(stocksymbols)


In [44]:
CustomStrategy = ta.Strategy(
    name="Defining Various TI to be used",
    description="SMA 10,50, BBANDS, RSI, MACD, STOCH, EMA 20,50,100",
    ta=[
        {"kind": "sma", "length": 10},
        {"kind": "sma", "length": 50},
        {"kind": "bbands", "length": 20},
        {"kind": "rsi"},
        {"kind": "macd", "fast": 8, "slow": 21},
        {"kind": "stoch"},
        {"kind": "ema", "length": 20},
        {"kind": "ema", "length": 50},
        {"kind": "ema", "length": 100},
    ]
)
df.ta.strategy(CustomStrategy)

In [ ]:
df.head(200)

In [ ]:
plt.plot(df['SMA_10'], label='SMA', color='green')
plt.plot(df['Open'], label='SHARE PRICE', color='black')

#add legend
plt.legend(title='Group')

#add axes labels and a title
plt.ylabel('SHARE PRICE', fontsize=14)
plt.xlabel('Time', fontsize=14)
plt.title('TA', fontsize=16)

#display plot
plt.show()

In [46]:
def buy_sell(data):
    signalBuy = []
    signalSell = []
    position = False 

    for i in range(len(data)):
        if data['SMA_10'][i] > data['SMA_50'][i]:
            if position == False :
                signalBuy.append(data['Adj Close'][i])
                signalSell.append(np.nan)
                position = True
            else:
                signalBuy.append(np.nan)
                signalSell.append(np.nan)
        elif data['SMA_10'][i] < data['SMA_50'][i]:
            if position == True:
                signalBuy.append(np.nan)
                signalSell.append(data['Adj Close'][i])
                position = False
            else:
                signalBuy.append(np.nan)
                signalSell.append(np.nan)
        else:
            signalBuy.append(np.nan)
            signalSell.append(np.nan)
    return pd.Series([signalBuy, signalSell])

In [47]:
df['Buy_Signal_price'], df['Sell_Signal_price'] = buy_sell(df)


In [ ]:
bs=df['Buy_Signal_price']
ss=df['Sell_Signal_price']
b = bs[np.logical_not(np.isnan(bs))]
s = ss[np.logical_not(np.isnan(ss))]
print(b)
print(s)

In [49]:
tp=0
fl=0
for i in range(len(s)):
  tp = tp + s[i]
  tp = tp - b[i]
print(tp)

43.48046875
